In [ ]:
import os
import shutil
import csv

csv_file = r"info.csv"
images_folder = r"volumes_without_annotations\sagittal_exports"
output_folder = r"Output"

normal_folder = os.path.join(output_folder, "Normal")
abnormal_folder = os.path.join(output_folder, "Abnormal")
os.makedirs(normal_folder, exist_ok=True)
os.makedirs(abnormal_folder, exist_ok=True)

with open(csv_file, newline='') as f:
    reader = csv.reader(f, delimiter=';')  
    for row in reader:
        if len(row) < 4 or not row[3].strip():
            print(f"Skipping row (no group): {row}")
            continue

        img_name = row[0] + "_sagittal.png" 
        group = row[3].strip().upper()     

        src_path = os.path.join(images_folder, img_name)
        if not os.path.exists(src_path):
            print(f"Image not found: {img_name}")
            continue

        if group == "G":
            dst_path = os.path.join(normal_folder, img_name)
        elif group in ["M", "I", "RIF"]:
            dst_path = os.path.join(abnormal_folder, img_name)
        else:
            print(f"Unknown group '{group}' for image {img_name}, skipping.")
            continue

        shutil.copy2(src_path, dst_path)  # copy image
        print(f"Copied {img_name} → {'Normal' if group=='G' else 'Abnormal'}")

print("All images have been organized into Normal and Abnormal folders.")


Image not found: Volume name_sagittal.png
Image not found: 04_01_sagittal.png
Image not found: 04_02_sagittal.png
Image not found: 04_03_sagittal.png
Image not found: 04_04_sagittal.png
Image not found: 04_05_sagittal.png
Image not found: 04_06_sagittal.png
Image not found: 04_07_sagittal.png
Image not found: 04_08_sagittal.png
Image not found: 04_09_sagittal.png
Image not found: 04_10_sagittal.png
Image not found: 04_11_sagittal.png
Image not found: 04_12_sagittal.png
Image not found: 04_13_sagittal.png
Image not found: 04_14_sagittal.png
Image not found: 04_15_sagittal.png
Image not found: 04_16_sagittal.png
Image not found: 04_17_sagittal.png
Image not found: 04_18_sagittal.png
Image not found: 04_19_sagittal.png
Image not found: 04_20_sagittal.png
Image not found: 04_21_sagittal.png
Image not found: 04_22_sagittal.png
Image not found: 04_23_sagittal.png
Image not found: 04_24_sagittal.png
Image not found: 04_25_sagittal.png
Image not found: 04_26_sagittal.png
Copied 04_27_sagittal.

In [ ]:
def rename_images(folder, prefix):
    files = sorted(os.listdir(folder))
    count = 1
    
    for f in files:
        if f.lower().endswith((".png", ".jpg", ".jpeg")):
            new_name = f"{prefix}_{count:03d}.jpg"
            old_path = os.path.join(folder, f)
            new_path = os.path.join(folder, new_name)
            os.rename(old_path, new_path)
            count += 1

rename_images("output/normal", "normal")
rename_images("output/abnormal", "abnormal")

print("Renaming completed!")


Renaming completed!


In [ ]:
import cv2
import numpy as np

input_folders = {
    "normal": "output/normal",
    "abnormal": "output/abnormal"
}

output_base = "output/cropped"
os.makedirs(output_base, exist_ok=True) 

for label, folder in input_folders.items():
    output_folder = os.path.join(output_base, label)
    os.makedirs(output_folder, exist_ok=True) 

    files = sorted(os.listdir(folder))
    for f in files:
        if f.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder, f)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            mask = img > 0
            coords = np.argwhere(mask)
            
            if coords.size == 0:
                print(f"Skipping {f} (empty image)")
                continue

            y0, x0 = coords.min(axis=0)
            y1, x1 = coords.max(axis=0)

            cropped = img[y0:y1+1, x0:x1+1]
            cropped_resized = cv2.resize(cropped, (256, 256))

            save_path = os.path.join(output_folder, f)
            cv2.imwrite(save_path, cropped_resized)

    print(f"Processed {label} images → saved in {output_folder}")

print("Cropping and resizing completed!")


Processed normal images → saved in output/cropped\normal
Processed abnormal images → saved in output/cropped\abnormal
Cropping and resizing completed!


In [ ]:
input_root = "output/cropped"
output_root = "output/normalized"

os.makedirs(os.path.join(output_root, "normal"), exist_ok=True)
os.makedirs(os.path.join(output_root, "abnormal"), exist_ok=True)

for category in ["normal", "abnormal"]:
    input_folder = os.path.join(input_root, category)
    output_folder = os.path.join(output_root, category)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tif")):

            in_path = os.path.join(input_folder, filename)
            out_path = os.path.join(output_folder, filename)

            img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(" Could not load:", in_path)
                continue

            img = img.astype(np.float32)
            img_norm = img / 255.0
            img_out = (img_norm * 255).astype(np.uint8)

            cv2.imwrite(out_path, img_out)

            print("✔ Normalized:", in_path)

✔ Normalized: output/cropped\normal\normal_001.jpg
✔ Normalized: output/cropped\normal\normal_002.jpg
✔ Normalized: output/cropped\normal\normal_003.jpg
✔ Normalized: output/cropped\normal\normal_004.jpg
✔ Normalized: output/cropped\normal\normal_005.jpg
✔ Normalized: output/cropped\normal\normal_006.jpg
✔ Normalized: output/cropped\normal\normal_007.jpg
✔ Normalized: output/cropped\normal\normal_008.jpg
✔ Normalized: output/cropped\normal\normal_009.jpg
✔ Normalized: output/cropped\normal\normal_010.jpg
✔ Normalized: output/cropped\normal\normal_011.jpg
✔ Normalized: output/cropped\normal\normal_012.jpg
✔ Normalized: output/cropped\normal\normal_013.jpg
✔ Normalized: output/cropped\normal\normal_014.jpg
✔ Normalized: output/cropped\normal\normal_015.jpg
✔ Normalized: output/cropped\normal\normal_016.jpg
✔ Normalized: output/cropped\normal\normal_017.jpg
✔ Normalized: output/cropped\normal\normal_018.jpg
✔ Normalized: output/cropped\normal\normal_019.jpg
✔ Normalized: output/cropped\no

In [72]:
df_info = pd.read_csv("info.csv", delimiter=';')
df_existing = pd.read_csv(r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Uterus-Abnormality-Detection\Ultrasound-Based-Classification-of-Uterine-Abnormalities\data\updated_labels.csv")


In [73]:
print(df_info.columns)


Index(['Volume name', 'Contains annotation', 'US machine model',
       'Group G/M/I/RIF'],
      dtype='object')


In [74]:
df_info_group = df_info['Group G/M/I/RIF'].str.strip().str.upper()


In [78]:

df_info = pd.read_csv("info.csv", delimiter=';')
s = df_info['Group G/M/I/RIF'].str.strip().str.upper()

num_g = (s == 'G').sum()
num_non_g = s.notna().sum() - num_g
num_missing = s.isna().sum()

print(f"G: {num_g}")
print(f"non-G: {num_non_g}")
print(f"missing: {num_missing}")

labels = ['G'] * int(num_g)
non_g = s[(s != 'G') & s.notna()].tolist()
labels += non_g

n = len(df_existing)
if len(labels) < n:
    labels += [''] * (n - len(labels))
else:
    labels = labels[:n]

df_existing['label'] = labels
df_existing.to_csv("updated_labels.csv", index=False)
print(f"Mapped labels: ids 1..{int(num_g)} = 'G', ids {int(num_g)+1}..{int(num_g)+len(non_g)} = non-G")

labels = df_existing['label'].astype(str).str.strip().str.upper()

df_existing['unexplained_infertility'] = (labels == 'I').astype(int)
df_existing['previous_miscarriages'] = (labels == 'M').astype(int)

df_existing.drop(columns=['age', 'implantation_failure'], inplace=True, errors='ignore')

df_existing.to_csv("updated_labels.csv", index=False)
print("Updated unexplained_infertility & previous_miscarriages; dropped age and implantation_failure.")



G: 68
non-G: 60
missing: 0
Mapped labels: ids 1..68 = 'G', ids 69..128 = non-G
Updated unexplained_infertility & previous_miscarriages; dropped age and implantation_failure.


In [82]:
df = pd.read_csv(r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Uterus-Abnormality-Detection\Ultrasound-Based-Classification-of-Uterine-Abnormalities\data\Dataset.csv")
df.loc[:, 'label'] = df['label'].apply(lambda x: 0 if x == 'G' else 1)

df.to_csv("Dataset.csv", index=False)
